In [72]:
import random
import datetime
from faker import Faker
import json

fake = Faker()

# Function to generate merchant profiles
def generate_merchant_profile(merchant_id):
    return {
        "merchant_id": merchant_id,
        "business_name": fake.company(),
        "business_type": random.choice(["Electronics", "Fashion", "Groceries"]),
        "registration_date": fake.date_between(start_date="-5y", end_date="today"),
        "gst_status": random.choice([True, False]),
        "average_ticket_size": round(random.uniform(500, 10000), 2)
    }

# Function to generate normal transactions
def generate_normal_transactions(merchant_id, num_txns=80):
    transactions = []
    for _ in range(num_txns):
        txn = {
            "transaction_id": fake.uuid4(),
            "merchant_id": merchant_id,
            "timestamp": fake.date_time_between(start_date="-30d", end_date="now"),
            "amount": round(random.uniform(50, 500), 2),  # Normal transaction amounts
            "customer_id": fake.uuid4(),
            "device_id": fake.uuid4(),
            "customer_location": fake.city(),
            "payment_method": random.choice(["Credit Card", "Debit Card", "UPI", "Net Banking"]),
            "product_category": random.choice(["Electronics", "Clothing", "Groceries"]),
            "platform": random.choice(["Web", "Mobile"]),
            "velocity_flag": False,
            "amount_flag": False,
            "time_flag": False,
    
            "label": 0,  # Normal transaction
        }
        transactions.append(txn)
    return transactions


# Function to inject late-night trading fraud
def inject_late_night_trading(transactions, config):
    for txn in transactions:
        # Set the current date for generating timestamps
        today = datetime.datetime.now().date()
        
        # Generate a timestamp between 11 PM today and 4 AM the next day
        start_time = datetime.datetime.combine(today, datetime.time(23, 0))
        end_time = datetime.datetime.combine(today + datetime.timedelta(days=1), datetime.time(4, 0))
        
        txn['timestamp'] = fake.date_time_between_dates(datetime_start=start_time, datetime_end=end_time)
        txn['time_flag'] = True
        txn['label'] = 1  # Fraudulent transaction
    return transactions


# Function to inject high-velocity spikes fraud
def inject_high_velocity_spikes(transactions, config):
    timestamp = fake.date_time_between(start_date="-30d", end_date="now")
    for txn in transactions:
        txn['timestamp'] = timestamp
        txn['velocity_flag'] = True
        txn['label'] = 1  # Fraudulent transaction
    return transactions

# Function to inject customer concentration fraud
def inject_customer_concentration(transactions, config):
    concentration_customer_ids = [fake.uuid4() for _ in range(5)]  # 5 high-risk customers
    for txn in transactions:
        if txn['customer_id'] in concentration_customer_ids:
            txn['amount_flag'] = True
            txn['label'] = 1  # Fraudulent transaction
    return transactions

# Function to generate fraudulent transactions for a merchant
def generate_fraudulent_transactions(merchant_id, num_txns=20):
    late_night_count = 6
    high_velocity_count = 7
    concentration_count = 7

    late_night_fraud = inject_late_night_trading(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(late_night_count)],
        config=None
    )
    high_velocity_fraud = inject_high_velocity_spikes(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(high_velocity_count)],
        config=None
    )
    customer_concentration_fraud = inject_customer_concentration(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(concentration_count)],
        config=None
    )

    return late_night_fraud + high_velocity_fraud + customer_concentration_fraud

# Generate data for multiple merchants
num_merchants = 100
merchant_profiles = [generate_merchant_profile(f"M{1000 + i}") for i in range(num_merchants)]
all_transactions = []

for profile in merchant_profiles:
    merchant_id = profile["merchant_id"]
    normal_data = generate_normal_transactions(merchant_id, num_txns=80)
    fraud_data = generate_fraudulent_transactions(merchant_id, num_txns=20)

    # Combine normal and fraud data for this merchant
    all_transactions.extend(normal_data + fraud_data)

# Shuffle the combined dataset
random.shuffle(all_transactions)

# Display the first 5 transactions


# # Save to JSON
# with open("transaction_dataset.json", "w") as f:
#     json.dump(all_transactions, f, indent=4)

# print(f"Generated data for {num_merchants} merchants.")
# print(f"Total transactions generated: {len(all_transactions)}")
all_transactions

[{'transaction_id': '4b070b85-1f84-4cb8-b113-ceefa107649e',
  'merchant_id': 'M1025',
  'timestamp': datetime.datetime(2024, 11, 15, 1, 19, 34),
  'amount': 215.49,
  'customer_id': '174d4f19-c408-40b0-a605-c6dbaf661a1b',
  'device_id': '1c58be65-c7aa-47bb-8c0a-f67e4147c6ed',
  'customer_location': 'South Aaronburgh',
  'payment_method': 'Net Banking',
  'product_category': 'Clothing',
  'platform': 'Mobile',
  'velocity_flag': False,
  'amount_flag': False,
  'time_flag': False,
  'label': 0},
 {'transaction_id': '8a44ce71-2dcd-40be-8c05-594af0dadfbf',
  'merchant_id': 'M1040',
  'timestamp': datetime.datetime(2024, 11, 5, 23, 1, 43),
  'amount': 101.1,
  'customer_id': '77ae89f3-1e6d-43a7-bbea-b605f27b2b0e',
  'device_id': 'c116f041-64a9-4579-bab0-700a9c33c6d1',
  'customer_location': 'Heatherchester',
  'payment_method': 'Credit Card',
  'product_category': 'Electronics',
  'platform': 'Mobile',
  'velocity_flag': False,
  'amount_flag': False,
  'time_flag': False,
  'label': 0},
 

In [73]:
import pandas as pd
import json

# Convert the dataset to a DataFrame
df = pd.DataFrame(all_transactions)

# Ensure 'timestamp' column is converted to datetime and then formatted to ISO strings
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')

# Save the DataFrame to a JSON file
df.to_json("transaction_dataset.json", orient='records', lines=True)

# Print the DataFrame to verify
transactions=df
df 

df.to_csv("train.csv", index=False)
df

,transaction_id,merchant_id,timestamp,amount,customer_id,device_id,customer_location,payment_method,product_category,platform,velocity_flag,amount_flag,time_flag,label
0,4b070b85-1f84-4cb8-b113-ceefa107649e,M1025,2024-11-15T01:19:34,215.49,174d4f19-c408-40b0-a605-c6dbaf661a1b,1c58be65-c7aa-47bb-8c0a-f67e4147c6ed,South Aaronburgh,Net Banking,Clothing,Mobile,False,False,False,0
1,8a44ce71-2dcd-40be-8c05-594af0dadfbf,M1040,2024-11-05T23:01:43,101.10,77ae89f3-1e6d-43a7-bbea-b605f27b2b0e,c116f041-64a9-4579-bab0-700a9c33c6d1,Heatherchester,Credit Card,Electronics,Mobile,False,False,False,0
2,6e32d150-de51-42e4-b6a8-790050c55fe2,M1073,2024-11-14T20:12:19,411.83,88a62f54-1f95-4913-a12d-085c1412274f,dec389b4-875a-4b0f-b199-a3457d5f8b0d,Michaelburgh,UPI,Electronics,Mobile,False,False,False,0
3,2ef25388-68e9-4fc3-ac02-3da9e61e5218,M1062,2024-11-28T23:05:43,363.12,dad807a7-de71-4542-bfa6-b5c1c86eccc6,189c9cf4-fd57-4a0c-aeb9-c02ee2b1adbc,South Paul,Net Banking,Electronics,Mobile,False,False,True,1
4,124ed179-74c0-431f-819e-82b3bd2f8de2,M1093,2024-11-06T19:18:51,127.04,c4466c5e-ee96-4f02-919d-98c28ea3c405,cf76f8f6-d3ff-4a2b-8b15-1b347f3e7490,Lake Jeremy,Net Banking,Groceries,Mobile,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,669847af-6a1c-4451-aea7-a3b44e287e76,M1045,2024-11-28T23:27:19,199.09,545b75a8-c1e6-460b-af67-9edf70386375,0d480170-0628-4965-99b2-da1f8fa52a71,Reginahaven,Credit Card,Electronics,Web,False,False,True,1
9996,16cfcf5d-dd54-4790-be2d-f5266ed777ff,M1029,2024-11-02T12:01:12,293.91,9ebdf1de-b926-4030-829a-cb2ba14c81a9,5c896174-af0d-404e-ba16-954bb75a7ee1,Cartershire,Debit Card,Electronics,Web,False,False,False,0
9997,e6a65211-5e66-4c26-88c3-67195f66d25c,M1035,2024-10-31T15:06:25,188.88,ee08ff3b-97a1-4855-acf3-5e32fecccecf,aace00a2-e4d2-4c15-ac05-672133cc91b0,Lake Darlene,Credit Card,Groceries,Web,False,False,False,0
9998,5d052618-12ba-49e3-917b-1b40dd0bc5b9,M1036,2024-11-04T23:02:16,68.68,48a5d616-4fa5-46b6-a123-4e514acc649d,3e54d84f-844a-4507-8dc5-63992bbea745,Pennyburgh,Net Banking,Clothing,Mobile,False,False,False,0


In [74]:
import random
import datetime
from faker import Faker
import json

fake = Faker()

# Function definitions (same as provided for training data)

def generate_merchant_profile(merchant_id):
    return {
        "merchant_id": merchant_id,
        "business_name": fake.company(),
        "business_type": random.choice(["Electronics", "Fashion", "Groceries"]),
        "registration_date": fake.date_between(start_date="-5y", end_date="today"),
        "gst_status": random.choice([True, False]),
        "average_ticket_size": round(random.uniform(500, 10000), 2)
    }

def generate_normal_transactions(merchant_id, num_txns=40):
    transactions = []
    for _ in range(num_txns):
        txn = {
            "transaction_id": fake.uuid4(),
            "merchant_id": merchant_id,
            "timestamp": fake.date_time_between(start_date="-30d", end_date="now"),
            "amount": round(random.uniform(50, 500), 2),  # Normal transaction amounts
            "customer_id": fake.uuid4(),
            "device_id": fake.uuid4(),
            "customer_location": fake.city(),
            "payment_method": random.choice(["Credit Card", "Debit Card", "UPI", "Net Banking"]),
            "product_category": random.choice(["Electronics", "Clothing", "Groceries"]),
            "platform": random.choice(["Web", "Mobile"]),
            "velocity_flag": False,
            "amount_flag": False,
            "time_flag": False,
            "label": 0,  # Normal transaction
        }
        transactions.append(txn)
    return transactions

def inject_late_night_trading(transactions, config):
    for txn in transactions:
        today = datetime.datetime.now().date()
        start_time = datetime.datetime.combine(today, datetime.time(23, 0))
        end_time = datetime.datetime.combine(today + datetime.timedelta(days=1), datetime.time(4, 0))
        txn['timestamp'] = fake.date_time_between_dates(datetime_start=start_time, datetime_end=end_time)
        txn['time_flag'] = True
        txn['label'] = 1  # Fraudulent transaction
    return transactions

def inject_high_velocity_spikes(transactions, config):
    timestamp = fake.date_time_between(start_date="-30d", end_date="now")
    for txn in transactions:
        txn['timestamp'] = timestamp
        txn['velocity_flag'] = True
        txn['label'] = 1  # Fraudulent transaction
    return transactions

def inject_customer_concentration(transactions, num_high_risk_customers=None, config=None):
    """
    Inject customer concentration fraud into a subset of transactions.
    
    Parameters:
    - transactions: List of transactions to inject fraud into.
    - num_high_risk_customers: Number of high-risk customer IDs to use. Defaults to 10% of transactions.
    - config: Additional configuration (not used here but kept for extensibility).
    
    Returns:
    - Modified transactions list with fraudulent entries.
    """
    # Default number of high-risk customers to 10% of transactions, capped at 20
    if num_high_risk_customers is None:
        num_high_risk_customers = min(max(1, len(transactions) // 10), 20)
    
    # Generate high-risk customer IDs
    concentration_customer_ids = [fake.uuid4() for _ in range(num_high_risk_customers)]
    
    # Assign these high-risk customer IDs to transactions
    for i, txn in enumerate(transactions):
        # Assign a high-risk customer ID to some transactions
        txn['customer_id'] = concentration_customer_ids[i % len(concentration_customer_ids)]
        
        # Mark them as fraudulent
        txn['amount_flag'] = True
        txn['label'] = 1  # Fraudulent transaction
    
    return transactions



def generate_fraudulent_transactions(merchant_id, num_txns=10):
    late_night_count = 3
    high_velocity_count = 4
    concentration_count = 3

    late_night_fraud = inject_late_night_trading(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(late_night_count)],
        config=None
    )
    high_velocity_fraud = inject_high_velocity_spikes(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(high_velocity_count)],
        config=None
    )
    customer_concentration_fraud = inject_customer_concentration(
        [generate_normal_transactions(merchant_id, 1)[0] for _ in range(concentration_count)],
        config=None
    )

    return late_night_fraud + high_velocity_fraud + customer_concentration_fraud

# Generate test data for merchants
num_test_merchants = 30
test_merchant_profiles = [generate_merchant_profile(f"T{2000 + i}") for i in range(num_test_merchants)]
test_transactions = []

for profile in test_merchant_profiles:
    merchant_id = profile["merchant_id"]
    normal_data = generate_normal_transactions(merchant_id, num_txns=40)
    fraud_data = generate_fraudulent_transactions(merchant_id, num_txns=10)

    test_transactions.extend(normal_data + fraud_data)

# Shuffle the test dataset
random.shuffle(test_transactions)

# # Save to JSON (optional)
# with open("test_transaction_dataset.json", "w") as f:
#     json.dump(test_transactions, f, indent=4)

# print(f"Generated test data for {num_test_merchants} merchants.")
# print(f"Total test transactions generated: {len(test_transactions)}")
test_transactions


[{'transaction_id': '588b6898-840f-4ee3-bed7-d7bf718d2c5c',
  'merchant_id': 'T2009',
  'timestamp': datetime.datetime(2024, 11, 24, 4, 30, 32),
  'amount': 437.31,
  'customer_id': 'ebd0e1b3-e6b7-419c-acef-f0e839f8e019',
  'device_id': '5040059d-dbea-4e2d-9388-e6b4ee756c7a',
  'customer_location': 'New Dean',
  'payment_method': 'Debit Card',
  'product_category': 'Groceries',
  'platform': 'Mobile',
  'velocity_flag': True,
  'amount_flag': False,
  'time_flag': False,
  'label': 1},
 {'transaction_id': '7c80bcc3-ddd9-4eda-86ca-1e08c00ad6eb',
  'merchant_id': 'T2025',
  'timestamp': datetime.datetime(2024, 11, 21, 6, 2, 47),
  'amount': 109.7,
  'customer_id': '0b187c86-03cd-4afe-ad4c-9465ca538ba4',
  'device_id': '60efac09-30f8-490d-b242-bcb93c774763',
  'customer_location': 'Brendamouth',
  'payment_method': 'UPI',
  'product_category': 'Clothing',
  'platform': 'Mobile',
  'velocity_flag': False,
  'amount_flag': False,
  'time_flag': False,
  'label': 0},
 {'transaction_id': '7c7

In [75]:
import pandas as pd
import json

# Convert the dataset to a DataFrame
df = pd.DataFrame(test_transactions)

# Ensure 'timestamp' column is converted to datetime and then formatted to ISO strings
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%S')

# Save the DataFrame to a JSON file
df.to_json("test_transaction_dataset.json", orient='records', lines=True)

# Print the DataFrame to verify
test_transactions=df
test_transactions
df.to_csv("test.csv", index=False)
df

,transaction_id,merchant_id,timestamp,amount,customer_id,device_id,customer_location,payment_method,product_category,platform,velocity_flag,amount_flag,time_flag,label
0,588b6898-840f-4ee3-bed7-d7bf718d2c5c,T2009,2024-11-24T04:30:32,437.31,ebd0e1b3-e6b7-419c-acef-f0e839f8e019,5040059d-dbea-4e2d-9388-e6b4ee756c7a,New Dean,Debit Card,Groceries,Mobile,True,False,False,1
1,7c80bcc3-ddd9-4eda-86ca-1e08c00ad6eb,T2025,2024-11-21T06:02:47,109.70,0b187c86-03cd-4afe-ad4c-9465ca538ba4,60efac09-30f8-490d-b242-bcb93c774763,Brendamouth,UPI,Clothing,Mobile,False,False,False,0
2,7c7e892d-3bf0-4770-a748-990f9db1400f,T2023,2024-11-14T07:34:54,495.50,d7d34239-a500-4148-9e3f-68e0bc8ea90a,73b548e1-e86f-4cc0-aa83-c79e07492ea8,Lake Christopher,Credit Card,Clothing,Mobile,False,False,False,0
3,9269fe3b-8a0a-49bc-9ff9-039dade7b5e7,T2016,2024-11-02T10:46:59,477.71,354b0a70-3ca5-4abb-be11-96141b86ec07,da5b9799-5b35-4af7-8ceb-8ab523fad746,North Ethanview,Debit Card,Clothing,Mobile,False,False,False,0
4,2598debb-8946-4edb-a559-6b24c498db59,T2012,2024-10-29T23:41:28,203.52,e8f3ef0d-8193-4e56-a1f9-3d00c3b764b3,68f8d0b4-92e3-4e1e-ae50-70389ff0f363,Lindahaven,UPI,Clothing,Mobile,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2ffd91d5-ce03-489c-9033-57aba19e88d4,T2005,2024-11-18T10:55:27,158.05,30201313-ea03-4f0b-9868-67016c5b2bc2,005fae23-5432-4c97-aaea-67a062fc731c,Lake Jamesfurt,Credit Card,Groceries,Web,False,False,False,0
1496,f58331bb-65ce-4ebd-939b-eb810a8c5037,T2015,2024-11-29T02:57:54,282.22,04b0531a-91ae-4d19-a047-270e90fd315f,7d61bdbf-bd6b-4aab-9e01-51c02f58c835,Butlerton,Debit Card,Electronics,Mobile,False,False,True,1
1497,b65fee15-10e7-4c8c-a5df-b45ebd8ccb16,T2004,2024-11-22T19:18:00,237.25,15fd35e2-9dc0-49b7-9db1-a0a22f10d65f,450fa583-c67b-4d4d-a18c-68c46dd58ae0,West Joseville,Credit Card,Clothing,Web,False,False,False,0
1498,a1de6d21-b3e7-4613-b73b-1b06a95cb626,T2008,2024-11-19T11:53:59,356.32,c120193f-6421-4ad5-9cbd-8c512eaed85b,3636eb5a-6164-47a2-86f0-69f9f19ac8e2,Port Lisa,UPI,Groceries,Web,False,False,False,0
